In [2]:
# I AM USING YIFAN LI'S CODE TO CLEAN MY DATA IN ORDER TO HAVE A SIMILAR DATASET WHEN PERFORMING TESTS

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

# import data
jan23 = pd.read_csv("nyc_crashes_202301.csv")
# create a copy for cleaning
jan23_cleaning = jan23.copy()
# gives the dimensions of our data

# replace 0.0 with nan (on cleaning dataframe)
jan23_cleaning["LATITUDE"] = jan23["LATITUDE"].replace([0.0], np.nan)
jan23_cleaning["LONGITUDE"] = jan23["LONGITUDE"].replace([0.0], np.nan)

# determines the latitude and longitude given a street address
def location(address):
    # concatenate more information
    new_addr = address + ", New York, New York"
    # try to locate address
    try:
        geolocator = Nominatim(user_agent="my_request")
        location = geolocator.geocode(new_addr)
        return location.latitude, location.longitude
    # if cannot locate, return nan
    except:
        return np.nan, np.nan
# all crashes that have an off street name entry
has_off_street = jan23['OFF STREET NAME'].notnull()
# all crashes with no latitude entry
no_lat = jan23['LATITUDE'].isnull()
# all crashes with an off street entry but no latitude entry
has_street_no_lat = jan23_cleaning[has_off_street & no_lat]
count = 0
# for each entry 
for i in has_street_no_lat["OFF STREET NAME"]:
    # if location is found, update the latitude and longitude
    if location(i) is not None:
        lat, lng = location(i)
        jan23_cleaning["LATITUDE"][count] = lat  
        jan23_cleaning["LONGITUDE"][count] = lng
        # printing to show the updates
        print(jan23_cleaning["LATITUDE"][count],jan23_cleaning["LONGITUDE"][count])
    count+=1

# series with all crashes missing "ZIP CODE"
no_zip = jan23['ZIP CODE'].isnull()
# series with all crashes missing "BOROUGH"
no_borough = jan23['BOROUGH'].isnull()
# check if zipcode and borough are always missing together
print("Zip and Borough are always missing together? ", no_zip.equals(no_borough))
# creates a series containing reports that are missing zipcode but have latitude
no_zip_yes_lat_mask = no_zip & (jan23["LATITUDE"].notnull())
no_zip_yes_lat = jan23[no_zip_yes_lat_mask]
# imports for the following functions
from uszipcode import SearchEngine
import numpy as np
from typing import Union, List
# Note this is code from the class notes
# Given zipcode, computes the respective borough
def nyczip2borough(zips: pd.Series) -> pd.Series:
    zips = zips.values if isinstance(zips, pd.Series) else zips
    condlist = [
        (zips >= 10001) & (zips <= 10282),
        (zips >= 10301) & (zips <= 10314),
        (zips >= 10451) & (zips <= 10475),
        (zips >= 11004) & (zips <= 11109),
        (zips >= 11351) & (zips <= 11697),
        (zips >= 11201) & (zips <= 11256),
    ]
    choicelist = [
        "MANHATTAN",
        "STATEN ISLAND",
        "BRONX",
        "QUEENS",
        "QUEENS",
        "BROOKLYN",
    ]
    result = pd.Series(np.select(condlist, choicelist, default=np.nan))
    return result
sr = SearchEngine()
# find the missing zipcodes
zipcodes = [(int(sr.by_coordinates(lat, lng, radius=5)[0].zipcode)) for lat, lng in zip(no_zip_yes_lat['LATITUDE'], no_zip_yes_lat['LONGITUDE'])]
zipcodes = pd.Series(zipcodes)
# find the missing boroughs
boroughs = nyczip2borough(zipcodes)
# replace missing entry with found zipcode
jan23_cleaning.loc[no_zip_yes_lat_mask, "ZIP CODE"] = zipcodes.values
# replace missing entry with found borough
jan23_cleaning.loc[no_zip_yes_lat_mask, "BOROUGH"] = boroughs.values
# printing to show that it works
print(jan23_cleaning.loc[no_zip_yes_lat_mask, "ZIP CODE"])
print(jan23_cleaning.loc[no_zip_yes_lat_mask, "BOROUGH"])

with pd.option_context('display.max_rows', None):
  print(jan23["CONTRIBUTING FACTOR VEHICLE 1"].value_counts(dropna=False))
with pd.option_context('display.max_rows', None):
  print(jan23["CONTRIBUTING FACTOR VEHICLE 1"].str.lower().value_counts(dropna=False))

/var/folders/4b/z0w6x5_n59g_x_9s3qnbx_p80000gn/T/ipykernel_11089/2844027849.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan23_cleaning["LATITUDE"][count] = lat
/var/folders/4b/z0w6x5_n59g_x_9s3qnbx_p80000gn/T/ipykernel_11089/2844027849.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan23_cleaning["LONGITUDE"][count] = lng


40.7167585 -73.45206739781105
nan nan
40.7653535 -73.9835992
40.887709 -73.906335
40.60449493877552 -73.75581197959184
nan nan
nan nan
40.877996207260004 -73.91660704983441
40.7972178 -73.9756094
40.718114 -74.015194
40.6790237 -73.863257
nan nan
40.9279045 -73.8565832857143
40.8160186 -73.9307271
40.68103655 -73.86556825
40.7343636 -73.88236725010495
40.6549829 -73.8727727
40.605943 -73.651673
40.72372565 -73.79011531221127
nan nan
40.78883414285715 -73.7009268095238
40.71302721739131 -73.98374686956522
nan nan
nan nan
nan nan
nan nan
40.6204791 -74.1774709
40.607888849999995 -74.0244909230886
nan nan
nan nan
40.7385509 -73.715183
Zip and Borough are always missing together?  True
0       11239.0
3       10306.0
4       11369.0
6       11413.0
8       10024.0
         ...   
6675    11101.0
6679    10459.0
6680    11205.0
6681    11418.0
6683    97003.0
Name: ZIP CODE, Length: 1880, dtype: float64
0            BROOKLYN
3       STATEN ISLAND
4              QUEENS
6              QUEENS


In [5]:
jan23.isnull().sum()

CRASH DATE                          0
CRASH TIME                          0
BOROUGH                          2270
ZIP CODE                         2270
LATITUDE                          448
LONGITUDE                         448
LOCATION                          448
ON STREET NAME                   1735
CROSS STREET NAME                3489
OFF STREET NAME                  4949
NUMBER OF PERSONS INJURED           0
NUMBER OF PERSONS KILLED            0
NUMBER OF PEDESTRIANS INJURED       0
NUMBER OF PEDESTRIANS KILLED        0
NUMBER OF CYCLIST INJURED           0
NUMBER OF CYCLIST KILLED            0
NUMBER OF MOTORIST INJURED          0
NUMBER OF MOTORIST KILLED           0
CONTRIBUTING FACTOR VEHICLE 1      51
CONTRIBUTING FACTOR VEHICLE 2    1713
CONTRIBUTING FACTOR VEHICLE 3    6062
CONTRIBUTING FACTOR VEHICLE 4    6513
CONTRIBUTING FACTOR VEHICLE 5    6631
COLLISION_ID                        0
VEHICLE TYPE CODE 1               125
VEHICLE TYPE CODE 2              2463
VEHICLE TYPE

In [6]:
jan23_cleaning.isnull().sum()

CRASH DATE                          0
CRASH TIME                          0
BOROUGH                           390
ZIP CODE                          390
LATITUDE                          534
LONGITUDE                         534
LOCATION                          448
ON STREET NAME                   1735
CROSS STREET NAME                3489
OFF STREET NAME                  4949
NUMBER OF PERSONS INJURED           0
NUMBER OF PERSONS KILLED            0
NUMBER OF PEDESTRIANS INJURED       0
NUMBER OF PEDESTRIANS KILLED        0
NUMBER OF CYCLIST INJURED           0
NUMBER OF CYCLIST KILLED            0
NUMBER OF MOTORIST INJURED          0
NUMBER OF MOTORIST KILLED           0
CONTRIBUTING FACTOR VEHICLE 1      51
CONTRIBUTING FACTOR VEHICLE 2    1713
CONTRIBUTING FACTOR VEHICLE 3    6062
CONTRIBUTING FACTOR VEHICLE 4    6513
CONTRIBUTING FACTOR VEHICLE 5    6631
COLLISION_ID                        0
VEHICLE TYPE CODE 1               125
VEHICLE TYPE CODE 2              2463
VEHICLE TYPE